- test-folder
    - category_1
        - manual_1
        - product_1
            - accessory_1
                - manual_4
            - manual_2
            - manual_3
        - product_2
            - manual_5
            - manual_6
            - manual_7
    - category_2
        - manual_8
        - product_3
            - accessory_2
                - manual_11
                - manual_12
            - manual_9
            - manual_10

#### Under folder MaxiCharger AC Ultra User Manual there are 2 pdf-files and 1 txt-files,
#### only 1 pdf-file that edited on 2023-07-06 should be verified.
#### The MaxiCharger DC Fast CMS Quick Guide is missing.
#### In summary, 15 documents shall be available, but only 14 documents need to be found via this function.

In [50]:
import re

def path_matcher (root):
    pattern = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(MaxiCharger (AC|DC))\\(.*)\\(.+ (Guide|Manual))$"
    match = re.match(pattern, root)
    category = match.group(3)
    object = match.group(5).replace("\\", " ")
    document = match.group(6)
    return category, object, document

c, o, d = path_matcher (r"target-folder\English_EN\MaxiCharger AC\Ultra\User Manual")
print(c, o, d)

MaxiCharger AC Ultra User Manual


In [47]:
import os
import re

verif = False
doc_num = 0
fund_num = 0

for root, dirs, files in os.walk(os.path.join('target-folder', 'English_EN')):
    if "arch" in dirs:
        doc_num += 1
        print(root, "...", dirs, "...", files)
        path_pattern = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(MaxiCharger (AC|DC))\\(.*)\\(.+ (Guide|Manual))$"
        path_match = re.match(path_pattern, root)
        category = path_match.group(3)
        object = path_match.group(5).replace("\\", " ")
        document = path_match.group(6)
        if len(files)!=0:
            fund_num += 1
            print("{0} {1} {2} is verified.".format(category, object, document))
        else:
            print("{0} {1} {2} is missing.".format(category, object, document))
    else:
        pass

print("%d documents shall be available." % (doc_num))
print("%d documents has been found." % (fund_num))

target-folder\English_EN\MaxiCharger AC\Energy Management System\Installation and Operation Manual ... ['arch'] ... ['Energy Management System - Operation and Installation Manual EU_20230525_EN.pdf']
MaxiCharger AC Energy Management System Installation and Operation Manual is verified.
target-folder\English_EN\MaxiCharger AC\Compact\User Manual ... ['arch'] ... ['MaxiCharger AC Compact - User Manual_20230302_EN.pdf']
MaxiCharger AC Compact User Manual is verified.
target-folder\English_EN\MaxiCharger AC\Compact\Quick Guide ... ['arch'] ... ['MaxiCharger AC Compact - Quick Guide_20230215_EN.pdf']
MaxiCharger AC Compact Quick Guide is verified.
target-folder\English_EN\MaxiCharger AC\Ultra\User Manual ... ['arch'] ... ['MaxiCharger AC Ultra - User Manual EU_20230621_EN.pdf', 'MaxiCharger AC Ultra - User Manual EU_20230706_EN.pdf', 'MaxiCharger AC Ultra - User Manual EU_20230712_EN.txt']
MaxiCharger AC Ultra User Manual is verified.
target-folder\English_EN\MaxiCharger AC\Ultra\Quick Guid

In [46]:

files = [
    ]
try:
    latest_file = max([file for file in files if file.endswith('.pdf')], key=lambda f: os.path.getmtime(os.path.join('target-folder\\English_EN\\MaxiCharger AC\\Ultra\\User Manual', f)))
except:
    latest_file = ""

latest_file

''

In [32]:
import os
import csv
import re

from datetime import datetime

# 目标文件夹根目录
target_folder = "target-folder"  

# 获取所有语言分类子文件夹的路径
#lang_folders = [folder for folder in os.listdir(target_folder) if os.path.isdir(os.path.join(target_folder, folder))]
#lang_folders.remove('English_EN')

lang_folders = ['English_EN']
lang_folder = lang_folders[0]
lang_code = re.match(r"([A-Z][a-zA-Z]+)_(\w{2})", lang_folder).group(2)  # EN

csv_file = f"{lang_code}_log.csv"  # CSV文件名
csv_path = os.path.join(target_folder, lang_folder, csv_file)

"""
"""
doc_num = 0
fund_num = 0
# 打开CSV文件并写入表头
with open(csv_path, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["language", "category", "object", "document", "release"])
    # 遍历目标文件夹下的所有内容
    for root, dirs, files in os.walk(os.path.join(target_folder, lang_folder)):
        # 只处理包含arch子项的文件夹
        if "arch" in dirs:
            doc_num += 1
            # 提取路径信息
            path_pattern = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(MaxiCharger (AC|DC))\\(.*)\\(.+ (Guide|Manual))$"
            path_match = re.match(path_pattern, root)
            category = path_match.group(3)
            object = path_match.group(5).replace("\\", " ")
            document = path_match.group(6)
            if len(files)!=0:
                fund_num += 1
                
                """
                # 获取目标文件版本信息
                file_pattern = r"(.*)_(\d{4})(\d{2})(\d{2})_(\w{2}).pdf"
                try:
                    # 目标为最新改动过的pdf后缀文件
                    latest_file = max([file for file in files if file.endswith('.pdf')], key=lambda f: os.path.getmtime(os.path.join(root, f)))
                    # 提取文档版本信息
                    file_match = re.match(file_pattern, latest_file)
                    release = f"{file_match.group(2)}-{file_match.group(3)}-{file_match.group(4)}"
                    #lang_code = file_match.group(5)
                except:
                    latest_file = 'null'
                    release = 'null'
                """
        else:
            pass
            
        #print(root)
        print(category)
        print(object)
        print(document)
        #print(latest_file)
        #print(release)
        print('='*30)

        #writer.writerow([lang_code, category, object, document, release])

print("%d documents shall be available." % (doc_num))
print("%d documents has been found." % (fund_num))
        


MaxiCharger DC
Fast CMS
Quick Guide
MaxiCharger DC
Fast CMS
Quick Guide
MaxiCharger DC
Fast CMS
Quick Guide
MaxiCharger AC
Energy Management System
Installation and Operation Manual
MaxiCharger AC
Energy Management System
Installation and Operation Manual
MaxiCharger AC
Energy Management System
Installation and Operation Manual
MaxiCharger AC
Compact
User Manual
MaxiCharger AC
Compact
User Manual
MaxiCharger AC
Compact
Quick Guide
MaxiCharger AC
Compact
Quick Guide
MaxiCharger AC
Compact
Quick Guide
MaxiCharger AC
Ultra
User Manual
MaxiCharger AC
Ultra
User Manual
MaxiCharger AC
Ultra
Quick Guide
MaxiCharger AC
Ultra
Quick Guide
MaxiCharger AC
Ultra
Quick Guide
MaxiCharger AC
Wallbox
User Manual
MaxiCharger AC
Wallbox
User Manual
MaxiCharger AC
Wallbox
Quick Guide
MaxiCharger AC
Wallbox
Quick Guide
MaxiCharger AC
Wallbox
Quick Guide
MaxiCharger AC
Wallbox Pedestal
Quick Guide
MaxiCharger AC
Wallbox Pedestal
Quick Guide
MaxiCharger AC
Wallbox Pedestal
Quick Guide
MaxiCharger AC
Wallbox 

In [132]:
files = ['f1.pdf', 'f2.exe']
print([f for f in files if f.endswith('.pdf')])

['f1.pdf']


In [6]:
import re

filename = "Maxicharger AC - Maintenance Manual EU_20220224_EN.pdf"
pattern = r"(.*)_(\d{4})(\d{2})(\d{2})_(\w+).pdf"

match = re.match(pattern, filename)

if match:
    release = f"{match.group(2)}-{match.group(3)}-{match.group(4)}"
    language_code = match.group(5)
    print("Release:", release)
    print("Language:", language_code)
else:
    print("No match found.")

Release: 2022-02-24
Language: EN


In [21]:
import re

string = "target-folder\English_EN\MaxiCharger AC\Energy Management System\Installation and Operation Manual"
#string = "target-folder\English_EN\MaxiCharger AC\Maintenance Manual"
# 当object不存在时，该如何匹配？
pattern = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(MaxiCharger (AC|DC))\\(.*)\\(.+ (Guide|Manual))$"

match = re.match(pattern, string)

if match:
    language = match.group(1)
    lang_code = match.group(2)
    category = match.group(3)
    object = match.group(5).replace("\\", " ")
    document = match.group(6)
    print(language)
    print(lang_code)
    print(category)
    print(object)
    print(document)
else:
    print("No match found.")

English
EN
MaxiCharger AC
Energy Management System
Installation and Operation Manual


In [ ]:
# 遍历每个语言分类子文件夹
for lang_folder in language_folders:
    lang_code = lang_folder.split("_")[-1]  # 提取语言代码 EN, FR, DE

    csv_file = f"{lang_code}_verification.csv"  # CSV文件名
    csv_path = os.path.join(target_folder, lang_folder, csv_file)

    # 打开CSV文件并写入表头
    with open(csv_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["language", "category", "object", "document", "update", "status"])

        # 遍历每个文档名称文件夹
        for root, dirs, files in os.walk(os.path.join(target_folder, lang_folder)):    
            # 'target-folder\\English_EN'
            # ('target-folder\\English_EN', ['MaxiCharger AC', 'MaxiCharger DC'], ['EN_verification.csv'])
            for folder in dirs:
                if "arch" in folder:  # 只处理包含arch子文件夹的文档名称文件夹
                    doc_folder = os.path.join(root, folder)
                    doc_name = folder.replace("arch", "").strip()  # 提取文档名称
                    product_name = os.path.basename(os.path.dirname(doc_folder))  # 提取产品名称
                    product_category = os.path.basename(os.path.dirname(os.path.dirname(doc_folder)))  # 提取产品分类

                    pdf_files = [file for file in files if file.lower().endswith(".pdf")]
                    if pdf_files:
                        latest_date = max([datetime.strptime(file.split("_")[2], "%Y%m%d") for file in pdf_files])
                        status = "up-to-date" if lang_folder == "English_EN" else "out-of-date"
                        latest_date_str = datetime.strftime(latest_date, "%Y%m%d")
                    else:
                        latest_date_str = "null"
                        status = "null"

                    writer.writerow([lang_code, product_category, product_name, doc_name, latest_date_str, status])

In [19]:
os.listdir("material-folder")

['manuals', 'certification']

In [22]:
import datetime

# 当天日期
current_date = datetime.date.today().strftime("%Y%m%d")
# 语言变量
language = "EN"
# 文件名模板
filename_template = "file-name_{0}_{1}"
# 根据模板生成文件名
filename = filename_template.format(current_date, language)
print(filename)


file-name_20230702_EN


In [ ]:
import os

# 原始目录和目标目录
source_dir = "./test-folder/English_EN\MaxiCharger AC/Wallbox/User Manual"
target_dir = "./test-folder/English_EN\MaxiCharger AC/Wallbox/User Manual"

# 遍历源目录下的所有文件
for filename in os.listdir(source_dir):
    if filename.endswith(".pdf"):
        # 构建源文件路径和目标文件路径
        source_file = os.path.join(source_dir, filename)
        target_file = os.path.join(target_dir, filename)
        
        # 将目标文件路径中的文件名替换为指定的 filename
        target_file = os.path.join(target_dir, filename.replace(".pdf", "_" + filename))
        
        # 重命名文件
        os.rename(source_file, target_file)